In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.tree import export_graphviz
from sklearn.model_selection import KFold as kf
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score,train_test_split
import graphviz
# import pydotplus
import io
from scipy import misc

%matplotlib inline

import hyperopt
from catboost import Pool, CatBoostClassifier, cv

from scipy.stats import uniform, randint

from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [18]:
data = pd.read_csv("MergedData.csv")
data.shape



(1048575, 11)

In [19]:
data['weight'] = data['weight'].apply(np.int64)


In [20]:
data.shape

(1048575, 11)

In [21]:
data.isnull().sum()

code_module          0
code_presentation    0
final_result         0
assessment_type      0
weight               0
score                0
gender               0
imd_band             0
id_site              0
activity_type        0
sum                  0
dtype: int64

In [22]:
train, test = train_test_split(data, test_size = 0.2)
print("Training size: {}, Test size: {}".format(len(train), len(test)))

Training size: 838860, Test size: 209715


In [23]:
train.shape


(838860, 11)

In [24]:
feature_cols =['code_module', 'code_presentation', 'assessment_type', 
               'weight','score', 'gender', 'imd_band', 'id_site', 'activity_type', 'sum']

In [25]:
X_train = train[feature_cols]
y_train = train['final_result']

X_test = test[feature_cols]
y_test = test['final_result']

## Final Feature Selection

In [26]:
#feature importance
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(X_train, y_train)
# display the relative importance of each attribute
print(model.feature_importances_)
# feature ranking
#5,7,8,10,4,1,2,6,3,9 

[0.05060915 0.04759519 0.01119016 0.06244458 0.3463518  0.03251835
 0.22932718 0.13847617 0.01167062 0.06981679]


In [27]:
# feature_cols =['code_module', 'code_presentation', 'assessment_type', 
#                'weight','score', 'gender', 'imd_band', 'id_site', 'activity_type', 'sum']

feature_cols =['score', 'imd_band', 'id_site', 'sum', 'weight']
X_train = train[feature_cols]
y_train = train['final_result']

X_test = test[feature_cols]
y_test = test['final_result']

## Loading The data Globally

In [28]:
data=pd.read_csv('MergedData.csv')

## XGB Classifier

In [29]:
df=data.copy()
df=df.loc[:,['score', 'imd_band', 'id_site', 'sum', 'weight','final_result']]
X=df.drop('final_result',axis=1)
Y=df.final_result

In [30]:
# You can experiment with many other options here, using the same .fit() and .predict()
# methods; see http://scikit-learn.org
# This example uses the current build of XGBoost, from https://github.com/dmlc/xgboost
x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size = 0.2,random_state=47)
gbm = xgb.XGBClassifier(max_depth=10, n_estimators=100, learning_rate=0.1,n_jobs=-1).fit(x_train, y_train)
y_pred=gbm.predict(x_test)
ac=accuracy_score(y_test,y_pred)*100
print('The accureacy is: ',round(ac,2),'%')

The accureacy is:  87.8 %


## Decision Tree Classifier

In [31]:
df=data.copy()

In [32]:
df=df.loc[:,['score', 'imd_band', 'id_site', 'sum', 'weight','final_result']]

In [33]:
df.head()
X=df.drop('final_result',axis=1)
Y=df.final_result

In [34]:
dclf=DecisionTreeClassifier(criterion='entropy',max_depth=10,random_state=47)

In [35]:
x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size = 0.2,random_state=47)
clf=dclf.fit(x_train,y_train)
y_pred=clf.predict(x_test)
ac=accuracy_score(y_test,y_pred)*100
print('The accureacy is: ',round(ac,2),'%')

The accureacy is:  72.61 %


## KNN Classifier

In [36]:
kclf=KNeighborsClassifier()

In [37]:
x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size = 0.2,random_state=47)
klf=kclf.fit(x_train,y_train)
y_pred=klf.predict(x_test)
ac=accuracy_score(y_test,y_pred)*100
print('The accureacy is: ',round(ac,2),'%')

The accureacy is:  71.73 %


## RandomForest Classifier

In [38]:
rfc=RandomForestClassifier(criterion='entropy',max_depth=15, n_jobs=-1)
x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size = 0.2,random_state=47)
rlf=rfc.fit(x_train,y_train)
y_pred=rlf.predict(x_test)
ac=accuracy_score(y_test,y_pred)*100
print('The accureacy is: ',round(ac,2),'%')

The accureacy is:  87.24 %


## Catboosting

In [39]:
import hyperopt
from catboost import Pool, CatBoostClassifier, cv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [40]:
train_df = pd.read_csv("MergedData.csv")

df = pd.DataFrame(train_df, columns=['score', 'imd_band', 'id_site', 'sum', 'weight','final_result'])

# df.shape
train_df.shape

(1048575, 11)

In [41]:
df['weight'] = df['weight'].apply(np.int64)

In [42]:
df.head()

,score,imd_band,id_site,sum,weight,final_result
0,60,8,877011,23,10,2
1,60,8,877012,305,10,2
2,60,8,877015,26,10,2
3,60,8,877023,6,10,2
4,60,8,877025,91,10,2


In [43]:
x = df.drop('final_result',axis=1)
y = df.final_result

y.head()

0    2
1    2
2    2
3    2
4    2
Name: final_result, dtype: int64

In [44]:
#choose the features we want to train, just forget the float data
cate_features_index = np.where(x.dtypes != float)[0]

print(cate_features_index)

[0 1 2 3 4]


In [45]:
cat_features = list(range(0, x.shape[1]))
print(cat_features)

[0, 1, 2, 3, 4]


In [46]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=.85,random_state=1234)

In [47]:
model = CatBoostClassifier(iterations=300,eval_metric='Accuracy',use_best_model=True,random_seed=42)

In [48]:
model.fit(xtrain,ytrain,cat_features=cate_features_index,eval_set=(xtest,ytest))

0:	learn: 0.7967369	test: 0.8011152	best: 0.8011152 (0)	total: 1.78s	remaining: 8m 51s
1:	learn: 0.8008859	test: 0.8038236	best: 0.8038236 (1)	total: 3.3s	remaining: 8m 12s
2:	learn: 0.8013762	test: 0.8038872	best: 0.8038872 (2)	total: 4.53s	remaining: 7m 28s
3:	learn: 0.8018329	test: 0.8046310	best: 0.8046310 (3)	total: 5.71s	remaining: 7m 2s
4:	learn: 0.8012281	test: 0.8040207	best: 0.8046310 (3)	total: 6.89s	remaining: 6m 46s
5:	learn: 0.8025992	test: 0.8055783	best: 0.8055783 (5)	total: 8.09s	remaining: 6m 36s
6:	learn: 0.8027809	test: 0.8057055	best: 0.8057055 (6)	total: 9.33s	remaining: 6m 30s
7:	learn: 0.8030726	test: 0.8059598	best: 0.8059598 (7)	total: 10.9s	remaining: 6m 36s
8:	learn: 0.8030278	test: 0.8059598	best: 0.8059598 (7)	total: 12s	remaining: 6m 28s
9:	learn: 0.8029930	test: 0.8058263	best: 0.8059598 (7)	total: 13.2s	remaining: 6m 22s
10:	learn: 0.8038356	test: 0.8070152	best: 0.8070152 (10)	total: 14.4s	remaining: 6m 17s
11:	learn: 0.8039377	test: 0.8068181	best: 0.

93:	learn: 0.9151262	test: 0.9365936	best: 0.9365936 (93)	total: 1m 55s	remaining: 4m 12s
94:	learn: 0.9154460	test: 0.9364855	best: 0.9365936 (93)	total: 1m 56s	remaining: 4m 11s
95:	learn: 0.9155200	test: 0.9364855	best: 0.9365936 (93)	total: 1m 57s	remaining: 4m 10s
96:	learn: 0.9156378	test: 0.9365809	best: 0.9365936 (93)	total: 1m 59s	remaining: 4m 9s
97:	learn: 0.9156748	test: 0.9365364	best: 0.9365936 (93)	total: 2m	remaining: 4m 8s
98:	learn: 0.9158622	test: 0.9369369	best: 0.9369369 (98)	total: 2m 1s	remaining: 4m 6s
99:	learn: 0.9159508	test: 0.9371022	best: 0.9371022 (99)	total: 2m 2s	remaining: 4m 5s
100:	learn: 0.9159161	test: 0.9372930	best: 0.9372930 (100)	total: 2m 4s	remaining: 4m 4s
101:	learn: 0.9159980	test: 0.9373375	best: 0.9373375 (101)	total: 2m 5s	remaining: 4m 3s
102:	learn: 0.9163009	test: 0.9374201	best: 0.9374201 (102)	total: 2m 6s	remaining: 4m 2s
103:	learn: 0.9163447	test: 0.9378143	best: 0.9378143 (103)	total: 2m 7s	remaining: 4m
104:	learn: 0.9166409	t

183:	learn: 0.9222698	test: 0.9444074	best: 0.9444074 (182)	total: 3m 47s	remaining: 2m 23s
184:	learn: 0.9222866	test: 0.9445218	best: 0.9445218 (184)	total: 3m 48s	remaining: 2m 22s
185:	learn: 0.9222776	test: 0.9444074	best: 0.9445218 (184)	total: 3m 49s	remaining: 2m 20s
186:	learn: 0.9223102	test: 0.9443311	best: 0.9445218 (184)	total: 3m 50s	remaining: 2m 19s
187:	learn: 0.9224246	test: 0.9445981	best: 0.9445981 (187)	total: 3m 51s	remaining: 2m 18s
188:	learn: 0.9225402	test: 0.9446172	best: 0.9446172 (188)	total: 3m 53s	remaining: 2m 16s
189:	learn: 0.9226288	test: 0.9446935	best: 0.9446935 (189)	total: 3m 54s	remaining: 2m 15s
190:	learn: 0.9226490	test: 0.9447189	best: 0.9447189 (190)	total: 3m 55s	remaining: 2m 14s
191:	learn: 0.9227141	test: 0.9446871	best: 0.9447189 (190)	total: 3m 57s	remaining: 2m 13s
192:	learn: 0.9227489	test: 0.9447316	best: 0.9447316 (192)	total: 3m 58s	remaining: 2m 12s
193:	learn: 0.9227567	test: 0.9447570	best: 0.9447570 (193)	total: 3m 59s	remain

273:	learn: 0.9271201	test: 0.9518142	best: 0.9518142 (273)	total: 5m 36s	remaining: 31.9s
274:	learn: 0.9271560	test: 0.9518460	best: 0.9518460 (274)	total: 5m 37s	remaining: 30.7s
275:	learn: 0.9272087	test: 0.9518905	best: 0.9518905 (275)	total: 5m 38s	remaining: 29.4s
276:	learn: 0.9272828	test: 0.9518778	best: 0.9518905 (275)	total: 5m 39s	remaining: 28.2s
277:	learn: 0.9273041	test: 0.9519477	best: 0.9519477 (277)	total: 5m 41s	remaining: 27s
278:	learn: 0.9273344	test: 0.9519731	best: 0.9519731 (278)	total: 5m 42s	remaining: 25.8s
279:	learn: 0.9273894	test: 0.9520240	best: 0.9520240 (279)	total: 5m 43s	remaining: 24.5s
280:	learn: 0.9274174	test: 0.9520431	best: 0.9520431 (280)	total: 5m 44s	remaining: 23.3s
281:	learn: 0.9274365	test: 0.9520939	best: 0.9520939 (281)	total: 5m 46s	remaining: 22.1s
282:	learn: 0.9274600	test: 0.9521384	best: 0.9521384 (282)	total: 5m 47s	remaining: 20.9s
283:	learn: 0.9274690	test: 0.9521512	best: 0.9521512 (283)	total: 5m 48s	remaining: 19.6s
2